In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
# need this!
!pip install geocoder

    100% |████████████████████████████████| 71kB 5.9MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 6.5MB/s eta 0:00:01
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import geocoder

In [ ]:
# to download a google sheet as a csv 
# https://docs.google.com/spreadsheets/d/LONG_ID_STRING/export?format=csv&id=LONG_ID_STRING
# https://docs.google.com/spreadsheets/d/1xa0iLqYKz8x9Yc_rfhtmSOJQ2EGgeUVjvV4A8LsIaxY/export?format=csv&id=1xa0iLqYKz8x9Yc_rfhtmSOJQ2EGgeUVjvV4A8LsIaxY

In [12]:
# df = pd.read_csv("Crowd Estimates, 1.21.2017 - By City.csv", skiprows=9,  header=None, thousands=",")
df = pd.read_csv("Crowd Estimates, 1.21.2017 - Sheet1.csv", skiprows=9,  header=None, thousands=",")
col_names = ["place", "state", "country", "estimate_low", "estimate_high", "est3", "not121", "source", "source2", "source3", "a", "b", "c","d","e", "f", "g", "h", "i", "j", "k", "l"]
df.columns = col_names
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 940 entries, 0 to 939
Data columns (total 22 columns):
place            939 non-null object
state            669 non-null object
country          934 non-null object
estimate_low     866 non-null float64
estimate_high    864 non-null float64
est3             5 non-null float64
not121           21 non-null float64
source           891 non-null object
source2          356 non-null object
source3          74 non-null object
a                937 non-null float64
b                78 non-null object
c                140 non-null object
d                139 non-null float64
e                87 non-null float64
f                87 non-null float64
g                257 non-null float64
h                257 non-null float64
i                31 non-null float64
j                31 non-null float64
k                275 non-null float64
l                275 non-null float64
dtypes: float64(14), object(8)
memory usage: 161.6+ KB


In [256]:
df

,place,state,country,estimate_low,estimate_high,est3,not121,source,source2,source3
0,"Abilene, TX",TX,US,200.0,200.0,NaN,NaN,http://www.reporternews.com/story/news/local/2...,NaN,NaN
1,"Accident, MD",MD,US,54.0,54.0,NaN,NaN,Twitter; on-site witness,NaN,NaN
2,"Adak, AK",AK,US,10.0,10.0,NaN,NaN,adn.com,NaN,NaN
3,"Adrian, MI",MI,US,130.0,150.0,NaN,NaN,https://www.facebook.com/events/847360115406578/,http://www.lenconnect.com/news/20170122/marche...,NaN
4,"Ajo, AZ",AZ,US,250.0,250.0,NaN,NaN,https://www.facebook.com/plugins/post.php?href...,NaN,NaN
5,"Alameda, CA",CA,US,8.0,8.0,NaN,NaN,FB,NaN,NaN
6,"Alamosa, CO",CO,US,350.0,350.0,NaN,NaN,https://twitter.com/zepol/status/8232349739352...,NaN,NaN
7,"Albany, CA",CA,US,500.0,500.0,NaN,NaN,https://twitter.com/ben_silverman/status/82303...,Albany PD (reported via google form),NaN
8,"Albany, NY",NY,US,7000.0,9000.0,NaN,NaN,http://www.timesunion.com/local/article/Albany...,FB,NaN
9,"Alexandria, VA",VA,US,17.0,17.0,NaN,NaN,https://twitter.com/TinaPStacy/status/82322604...,NaN,NaN


In [8]:
# a few added here to fix data in spreadsheet
state_lookup = { 'AK': 'Alaska', 'AL': 'Alabama', 'AR': 'Arkansas', 'AS': 'American Samoa', 'AZ': 'Arizona', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DC': 'District of Columbia', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'GU': 'Guam', 'HI': 'Hawaii', 'IA': 'Iowa', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'MA': 'Massachusetts', 'MD': 'Maryland', 'ME': 'Maine', 'MI': 'Michigan', 'MN': 'Minnesota', 'MO': 'Missouri', 'MP': 'Northern Mariana Islands', 'MS': 'Mississippi', 'MT': 'Montana', 'NA': 'National', 'NC': 'North Carolina', 'ND': 'North Dakota', 'NE': 'Nebraska', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NV': 'Nevada', 'NY': 'New York', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'PR': 'Puerto Rico', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VA': 'Virginia', 'VI': 'Virgin Islands', 'VT': 'Vermont', 'WA': 'Washington', 'WI': 'Wisconsin', 'WV': 'West Virginia', 'WY': 'Wyoming', 'CA (Seacrest)':'California', 'Guam': 'Guam'}

In [13]:
# drop what we don't care about
df.drop(["est3", "not121"], axis=1, inplace=True)
# "a", "b", "c","d","e", "f", "g", "h", "i", "j", "k", "l"
df.drop(["a", "b", "c","d","e", "f", "g", "h", "i", "j", "k", "l"], axis=1, inplace=True)
# drop rows that lack a max estimate
df.dropna(axis=0, how='any', subset=['estimate_high'], inplace=True)
df

,place,state,country,estimate_low,estimate_high,source,source2,source3
2,"Abilene, TX",TX,US,200.0,200.0,http://www.reporternews.com/story/news/local/2...,NaN,NaN
3,"Accident, MD",MD,US,54.0,54.0,http://www.riverfronttimes.com/newsblog/2017/0...,NaN,Twitter; on-site witness
4,"Adak, AK",AK,US,10.0,10.0,adn.com,NaN,NaN
5,"Adrian, MI",MI,US,150.0,150.0,http://www.lenconnect.com/news/20170122/marche...,http://www.lenconnect.com/news/20170122/marche...,NaN
6,"Ajo, AZ",AZ,US,250.0,250.0,https://www.facebook.com/plugins/post.php?href...,NaN,NaN
7,"Alameda, CA",CA,US,8.0,8.0,FB,NaN,NaN
8,"Alamosa, CO",CO,US,350.0,350.0,https://twitter.com/zepol/status/8232349739352...,NaN,NaN
9,"Albany, CA",CA,US,500.0,500.0,https://twitter.com/ben_silverman/status/82303...,Albany PD (reported via google form),NaN
10,"Albany, NY",NY,US,7000.0,9000.0,http://www.timesunion.com/local/article/Albany...,FB,NaN
11,"Alexandria, VA",VA,US,17.0,17.0,https://twitter.com/TinaPStacy/status/82322604...,NaN,NaN


In [14]:
# this is spelled wrong
t_town = df[ df['place']=='Tblisi, Georgia']
df.loc[t_town.index,'place'] = 'Tbilisi, Georgia'

# this is that header type row
df.drop( df[ df.place.str.contains('partial')].index, axis=0, inplace=True )

In [15]:

def fix_us_place_names(place, country):
    if country != "US":
        return place
    res  = place.split(', ')
    if len(res) < 2 or res[-1] not in state_lookup:
        return place
    state = state_lookup[res[-1]]
    res[-1]= state
    res.append('United States')
    return ', '.join(res)

# z = "Abilene, TX"
# # z = "Encinitas, CA (Seacrest)"
# country = "US"
# fix_us_place_names(z, country)

# df['col'] = 'str' + df['col'].astype(str)
# df["new_column"] = df.apply(lambda x: x["A"] * x["B"], axis = 1)
# df.columns = df.columns.append('full_place')

# create a new col with the full place name, for easier geocoding
df = df.assign(full_place=df.place)
df.full_place = df.apply(lambda x: fix_us_place_names(x.place, x.country) , axis=1 )



In [ ]:
# g = geocoder.google("New York, New York, United States")
# g.latlng

In [16]:
cache = {}

In [17]:
# make our new cols
df = df.assign(latlng=df.place)
df = df.assign(lng=df.place)


def get_lat(x): 
    if x in cache:
        return cache[x][0]    
    print(x)
    g = geocoder.google(x)
    if g.latlng:
        cache[x] = g.latlng
        return g.latlng[0]
    else:
        return pd.np.nan

def get_lng(x): 
    if x in cache:
        return cache[x][1]
    print(x)
    g = geocoder.google(x)
    if g.latlng:
        cache[x] = g.latlng        
        return g.latlng[1]
    else:
        return pd.np.nan

df['lat'] = df.full_place.apply(lambda x: get_lat(x) )
df['lng'] = df.full_place.apply(lambda x: get_lng(x) )
df
# this can actually be run multiple times, which will fix missing data

Abilene, Texas, United States
Accident, Maryland, United States
Adak, Alaska, United States
Adrian, Michigan, United States
Ajo, Arizona, United States
Alameda, California, United States
Alamosa, Colorado, United States
Albany, California, United States
Albany, New York, United States
Alexandria, Virginia, United States
Albuquerque, New Mexico, United States
Alliance, Nebraska, United States
Almanor West, California, United States
Alpine, Texas, United States
Amarillo, Texas, United States
Amelia Island, Florida, United States
Anacortes, Washington, United States
Anchorage, Alaska, United States
Angola, Indiana, United States
Ann Arbor, Michigan, United States
Annapolis, Maryland, United States
Annville, Pennsylvania, United States
Appleton, Wisconsin, United States
Arden, Delaware, United States
Asbury Park, New Jersey, United States
Asheville, North Carolina, United States
Ashland, Oregon, United States
Aspen, Colorado, United States
Astoria, Oregon, United States
Athens, Georgia, Un

,place,state,country,estimate_low,estimate_high,source,source2,source3,full_place,latlng,lng,lat
2,"Abilene, TX",TX,US,200.0,200.0,http://www.reporternews.com/story/news/local/2...,NaN,NaN,"Abilene, Texas, United States","Abilene, TX",-99.733144,32.448736
3,"Accident, MD",MD,US,54.0,54.0,http://www.riverfronttimes.com/newsblog/2017/0...,NaN,Twitter; on-site witness,"Accident, Maryland, United States","Accident, MD",-79.319759,39.628696
4,"Adak, AK",AK,US,10.0,10.0,adn.com,NaN,NaN,"Adak, Alaska, United States","Adak, AK",-176.658056,51.880000
5,"Adrian, MI",MI,US,150.0,150.0,http://www.lenconnect.com/news/20170122/marche...,http://www.lenconnect.com/news/20170122/marche...,NaN,"Adrian, Michigan, United States","Adrian, MI",-84.037166,41.897547
6,"Ajo, AZ",AZ,US,250.0,250.0,https://www.facebook.com/plugins/post.php?href...,NaN,NaN,"Ajo, Arizona, United States","Ajo, AZ",-112.860710,32.371725
7,"Alameda, CA",CA,US,8.0,8.0,FB,NaN,NaN,"Alameda, California, United States","Alameda, CA",-122.241636,37.765206
8,"Alamosa, CO",CO,US,350.0,350.0,https://twitter.com/zepol/status/8232349739352...,NaN,NaN,"Alamosa, Colorado, United States","Alamosa, CO",-105.870021,37.469449
9,"Albany, CA",CA,US,500.0,500.0,https://twitter.com/ben_silverman/status/82303...,Albany PD (reported via google form),NaN,"Albany, California, United States","Albany, CA",-122.297747,37.886870
10,"Albany, NY",NY,US,7000.0,9000.0,http://www.timesunion.com/local/article/Albany...,FB,NaN,"Albany, New York, United States","Albany, NY",-73.756232,42.652579
11,"Alexandria, VA",VA,US,17.0,17.0,https://twitter.com/TinaPStacy/status/82322604...,NaN,NaN,"Alexandria, Virginia, United States","Alexandria, VA",-77.046921,38.804836


In [19]:
df.dropna(axis=0, how='any', subset=['lat', 'lng'], inplace=True)
# df.drop(["latlng"], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 858 entries, 2 to 938
Data columns (total 12 columns):
place            858 non-null object
state            624 non-null object
country          858 non-null object
estimate_low     858 non-null float64
estimate_high    858 non-null float64
source           858 non-null object
source2          353 non-null object
source3          74 non-null object
full_place       858 non-null object
latlng           858 non-null object
lng              858 non-null float64
lat              858 non-null float64
dtypes: float64(4), object(8)
memory usage: 87.1+ KB


In [20]:
df.to_json("marches.json", orient="records")

In [265]:

# df.info()
# cache['Madison, Wisconsin, United States']
# df [ df['full_place']== "Madison, Wisconsin, United States" ]
# df[df.lat == False]
# df[df.lng == False]